# Introduction

Today we’ll be exploring multivariate meteorological data from Mar Casado Beach, Sao Paulo (Brazil). By the end of the lesson, participants will be able to:

Create a database in a notebook using DuckDBClient.of()
Query the database in SQL cells
Visualize relationships between variables
Import and reuse existing content using import with
Perform and visualize results of principal component analysis
In this session, we will work with data from da Silva et al (2019) describing monthly atmospheric and ocean data (e.g. air pressure, windspeed, tides, sea surface temperature).

Data source: Marcos Gonçalves da Silva, Juliana Nascimento Silva, Helena Rodrigues Fragoso, Natalia Pirani Ghilardi-Lopes (2019). Temporal series analysis of abiotic data for a subtropical Brazilian rocky shore. Data in Brief, Volume 24. ISSN 2352-3409, https://doi.org/10.1016/j.dib.2019.103873.

Step 1: Combine the tables into a database
The data are already attached here in two parts: marCasadoAir, and marCasadoSea:


```{ojs}
marCasadoAir = FileAttachment("marCasadoAir@5.csv").csv({typed: true}) // Date issue
```

```{ojs}
marCasadoSea = FileAttachment("marCasadoSea@4.csv").csv({typed: true}) // Date issue
```


We can create a database containing both tables using DuckDBClient.of():


```{ojs}
// Write code to create a database called marCasadoDB, with tables 'air' and 'sea':
marCasadoDB = DuckDBClient.of({air: marCasadoAir, sea: marCasadoSea})
```


Now, open the Database pane in the right margin of your notebook. There, you can explore the schema and variables of your newly created database.

Step 2: Wrangle & analyze data in a SQL cell
We want to combine some data from both tables in the database. The column we’re able to join by is month. We will also keep the following columns, and add a new column for season:

From air:

* month
* meanPressure (mmHg)
* windSpeed (kilometers per hour)
* PAR (photoactive radiation in E/m s2)
* meanHumidity (percent)
* windDirection (degrees)

From sea:

* maxTide (meters)
* minTide (meters)
* salinity (practical salinity units)
* seaSurfaceTemp (degrees Celsius)

From da Silva et al (2019): “...two distinct seasons: (I) a hot and moist season from October to March (encompassing Spring and Summer) and (II) a cold and dry season from April to September (encompassing Autumn and Winter); an expected result for subtropical zones.”

We'll also add a new column, season, containing "cool dry" for October thru March, otherwise "hot moist."


```{sql}
select  a.month, 
        a.meanPressure, 
        a.windSpeed,
        a.PAR,
        a.meanHumidity, 
        a.windDirection,
        s.maxTide,
        s.minTide,
        s.salinity,
        s.seaSurfaceTemp,
        CASE WHEN date_part('month', a.month) IN (10, 11, 12, 1, 2, 3) THEN 'hot moist' ELSE 'cool dry' END AS season
from air as a
left join sea as s
on a.month = s.month
```